In [ ]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.5.1+cu121.html

In [ ]:
!pip install torch_geometric

In [3]:
from torch.optim import Adam
from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import TransE, ComplEx
from kg_model_utils import train

In [7]:
FB15k_237('data/FB15k', split='test')[0].edge_type

tensor([148,  92,  13,  ..., 177, 131,  26])

In [9]:
set(FB15k_237('data/FB15k', split='test')[0].edge_type.detach().cpu().numpy())

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 165,
 166,
 167,
 168,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 179,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,

In [10]:
train_data = FB15k_237('data/FB15k', split='train')[0]
val_data = FB15k_237('data/FB15k', split='val')[0]
test_data = FB15k_237('data/FB15k', split='test')[0]

In [11]:
train_data

Data(edge_index=[2, 272115], edge_type=[272115], num_nodes=14541)

In [13]:
val_data

Data(edge_index=[2, 17535], edge_type=[17535], num_nodes=14541)

In [14]:
train_data.num_nodes

14541

In [15]:
train_data.num_edge_types

237

In [16]:
model = TransE(num_nodes=train_data.num_nodes,
               num_relations=train_data.num_edge_types,
               hidden_channels=50)

In [17]:
model

TransE(14541, num_relations=237, hidden_channels=50)

In [18]:
test_data.edge_index

tensor([[ 6180,  1454,  6651,  ...,  1677, 12770,   452],
        [ 2861,  5951,   637,  ...,   589, 14420,  1098]])

In [19]:
loader = model.loader(head_index=train_data.edge_index[0],
                      rel_type=train_data.edge_type,
                      tail_index=train_data.edge_index[1],
                      batch_size=1000,
                      shuffle=True)

In [23]:
batch = next(iter(loader))

In [26]:
batch[0]

tensor([ 3950,  7750,  9534,   214,  9752,  1389,  4406,  6714,  7773,    53,
         2429,  8935,  3389,  2233,  6459, 12227,  2927,  8541,  5374,  7622,
          831,  2301, 10992, 11211,  9642,  1296,    26,  2019,  4066,  8471,
         4912,  8491,  2820,  9437, 11190,  6617,  5010,  1329,  2663,   978,
          206,  2520, 12113,  1159,  5191,   312,  3588,   637,  6157,  1476,
         7297, 11463,  8052,  7862,  3042, 10574,  1477,  6995,  6404,  9302,
         5106,   969,  2036,   212, 10342,  8410,  1121,  1711,  5136,   278,
         4418,  6056,  1121,  5315,    32,   906,   254,  5599,  3808,   559,
         4408,   593,  2524,  7468,  4953,  3464,  7456,  6483,  2433,  6082,
          961,  1148,  8219,  9904,  3179, 10394,  5104,  1654,  6492,  8229,
        12027,  1329,  7162,  4998,  7479,  5193, 14286,  6170,  1830,  2086,
          495,   570,  8849,  1987, 10544,   236, 13735,  3906,  9527,  1124,
         8244,  2466,   826,  3187,  1990,  3127,   268,  3477, 

In [25]:
batch[0].size()

torch.Size([1000])

In [27]:
optimizer = Adam(model.parameters(), lr=0.01)

In [29]:
train(model=model,
      data_loader=loader,
      optimizer=optimizer,
      epochs=10)

Epoch: 000, Loss: 0.7591
Epoch: 001, Loss: 0.5505
Epoch: 002, Loss: 0.4330
Epoch: 003, Loss: 0.3544
Epoch: 004, Loss: 0.3020
Epoch: 005, Loss: 0.2677
Epoch: 006, Loss: 0.2452
Epoch: 007, Loss: 0.2297
Epoch: 008, Loss: 0.2185
Epoch: 009, Loss: 0.2092


In [30]:
rank, mrr, hits10 = model.test(head_index=test_data.edge_index[0],
                               rel_type=test_data.edge_type,
                               tail_index=test_data.edge_index[1],
                               batch_size=1000, k=10)

100%|██████████| 20466/20466 [08:09<00:00, 41.80it/s]


In [31]:
hits10

0.34437603830743674